In [10]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
from delta.tables import *

StatementMeta(, aa33e45a-c536-4e19-a0dd-06f781905d58, 12, Finished, Available, Finished)

In [11]:
DeltaTable.createIfNotExists(spark)\
            .tableName("Gold_orderPriority")\
            .addColumn("OrderPriority_ID", LongType())\
            .addColumn("Order_Priority", StringType())\
            .addColumn("Created_TS", TimestampType())\
            .addColumn("Modified_TS", TimestampType())\
            .execute()

StatementMeta(, aa33e45a-c536-4e19-a0dd-06f781905d58, 13, Finished, Available, Finished)

In [13]:
df = spark.read.table("Ecommere_Lh.gold_orderpriority")

StatementMeta(, aa33e45a-c536-4e19-a0dd-06f781905d58, 15, Finished, Available, Finished)

In [14]:
max_date = df.selectExpr("coalesce(max(Modified_TS), '1900-01-01')").first()[0]

StatementMeta(, aa33e45a-c536-4e19-a0dd-06f781905d58, 16, Finished, Available, Finished)

In [15]:
max_date

StatementMeta(, aa33e45a-c536-4e19-a0dd-06f781905d58, 17, Finished, Available, Finished)

'1900-01-01'

In [16]:
df_bronze = spark.read.table("Ecommere_Lh.bronze_sales")

StatementMeta(, aa33e45a-c536-4e19-a0dd-06f781905d58, 18, Finished, Available, Finished)

In [23]:
from pyspark.sql.functions import col
df_bronze_mod = df_bronze.select("Order_Priority").where(col("Modified_TS")>max_date).drop_duplicates()

StatementMeta(, aa33e45a-c536-4e19-a0dd-06f781905d58, 25, Finished, Available, Finished)

In [24]:
df_bronze.select("Order_Priority").drop_duplicates().show()

StatementMeta(, aa33e45a-c536-4e19-a0dd-06f781905d58, 26, Finished, Available, Finished)

+--------------+
|Order_Priority|
+--------------+
|          High|
|           Low|
|        Medium|
|      Critical|
+--------------+



In [25]:
max_id = df.selectExpr("coalesce(max(OrderPriority_ID),0)").first()[0]

StatementMeta(, aa33e45a-c536-4e19-a0dd-06f781905d58, 27, Finished, Available, Finished)

In [26]:
max_id

StatementMeta(, aa33e45a-c536-4e19-a0dd-06f781905d58, 28, Finished, Available, Finished)

0

In [29]:
df_final = df_bronze_mod.withColumn("OrderPriority_ID", max_id+monotonically_increasing_id()+1)

StatementMeta(, aa33e45a-c536-4e19-a0dd-06f781905d58, 31, Finished, Available, Finished)

In [30]:
df_gold_delta = DeltaTable.forPath(spark,"Tables/gold_orderpriority")
df_bronze_table = df_final

StatementMeta(, aa33e45a-c536-4e19-a0dd-06f781905d58, 32, Finished, Available, Finished)

In [31]:
df_gold_delta.alias("gold")\
                .merge(\
                    df_bronze_table.alias("bronze"),\
                    "gold.Order_Priority==Bronze.Order_Priority"\
                )\
                .whenMatchedUpdate(\
                set = {
                        "gold.Modified_TS":current_timestamp()    
                    }
                )\
                .whenNotMatchedInsert(\
                    values={
                        "gold.OrderPriority_ID":"Bronze.OrderPriority_ID",
                        "gold.Order_Priority":"Bronze.Order_Priority",
                        "gold.Created_TS":current_timestamp(),
                        "gold.modified_TS":current_timestamp()
                    }
                )\
                .execute()

StatementMeta(, aa33e45a-c536-4e19-a0dd-06f781905d58, 33, Finished, Available, Finished)

In [32]:
%%sql
SELECT * from gold_orderpriority

StatementMeta(, aa33e45a-c536-4e19-a0dd-06f781905d58, 34, Finished, Available, Finished)

<Spark SQL result set with 4 rows and 4 fields>